In [1]:

%matplotlib widget
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from IPython.display import display
import os
import sys
from datetime import date
import json
import numpy as np
#print(os.getcwd())

In [2]:
ET_DATA_PATH = 'D:\_MasterArbeit_EEDA\EyeTracking'
SCENE_EVENT_DATA_PATH = 'D:\_MasterArbeit_EEDA\SceneData'
STEERING_DATA_PATH = 'D:\_MasterArbeit_EEDA\Input'
PARTICIPANT_DATA_PATH = 'D:\_MasterArbeit_EEDA\ParticipantCalibrationData'
PROCESSED_DATA_PATH = '..\csv_et'
PLOT_PATH = '..\PLOTS\data_quality'
os.makedirs(os.path.dirname(PROCESSED_DATA_PATH),exist_ok=True)
os.makedirs(os.path.dirname(PLOT_PATH), exist_ok=True)

### Filter EyeValidationError under 1.5

In [3]:
df = pd.DataFrame()

files = glob.glob(f'{PARTICIPANT_DATA_PATH}/*.txt')

for fi in files[:4]: # reading all files
    with open(fi, 'r') as j:
        tmp = json.loads(j.read())
        tempdf0 = pd.DataFrame.from_dict(tmp,orient='index').T        
        tempdf0[["EyeValidationError.x","EyeValidationError.y","EyeValidationError.z"]] = pd.DataFrame(tempdf0["EyeValidationError"].to_list(), columns=['x', 'y','z'])
        if(tempdf0["EyeValidationError.x"][0]<1.5) and (tempdf0["EyeValidationError.y"][0]<1.5):
            df = pd.concat([df, tempdf0], ignore_index=True)
        
df = df.drop(['IPAddress', 'VRmode',
       'SteeringInputDevice', 'EyeValidationError', 'SeatCalibrationOffset',
       'TrainingDuration', 'ExperimentDuration', 'ApplicationDuration',
       'AverageExperimentFPS', 'SpecialNotes', 'EyeValidationError.z'], axis=1)
display(df)

,ParticipantUuid,ExperimentalCondition,EyeValidationError.x,EyeValidationError.y
0,489749a7e3394cdda22f5f2717a340a8,ManualDriving,0.516909,0.679336


### Check if Incomplete

In [4]:
files = glob.glob(f'{ET_DATA_PATH}/*.txt')
des_index = []
for i in range(0,df['ParticipantUuid'].count()):
    for fi in files[:]: # reading all files
        #print (df['ParticipantUuid'][i])
        if '_Incomplete' in fi:
            if df['ParticipantUuid'][i] in fi:
                if os.path.getsize(fi) < 10 : 
                    des_index.append(i)
                    
                
df = df.iloc[des_index] 
df = df.reset_index(drop=True)
display(df)

,ParticipantUuid,ExperimentalCondition,EyeValidationError.x,EyeValidationError.y
0,489749a7e3394cdda22f5f2717a340a8,ManualDriving,0.516909,0.679336


### Load Eyetracking Data

In [5]:
files = glob.glob(f'{ET_DATA_PATH}/*.txt')
for i in range(0,df['ParticipantUuid'].count()):
    et_df = pd.DataFrame()
    for fi in files[:]: # reading all files
        if df['ParticipantUuid'][i] in fi:
            with open(fi, 'r') as j:
                tmp = json.loads(j.read())
            for nS, sample in enumerate(tmp):
                tmpdf = pd.json_normalize(data=sample, sep='_')
                tmpdf['uid'] = fi.split('\\')[-1].split('_')[0]
                tmpdf['scene'] = fi.split('_')[-1].split('.')[0]
                tmpdf = tmpdf.drop(['FPS','pupilDiameterMillimetersLeft', 'pupilDiameterMillimetersRight',
       'EyeBlinkingWorldLeftTobii', 'EyeBliningWorldRightTobii',
       'EyeBlinkingLocalLeftTobii', 'EyeBlinkingLocalRightTobii', 'hitObjects',
       'HmdRotation_x', 'HmdRotation_y',
       'HmdRotation_z', 'HmdRotation_w','EyePositionLeftWorld_x',
       'EyePositionLeftWorld_y', 'EyePositionLeftWorld_z',
       'EyeDirectionLeftWorld_x', 'EyeDirectionLeftWorld_y',
       'EyeDirectionLeftWorld_z', 'EyePositionLeftLocal_x',
       'EyePositionLeftLocal_y', 'EyePositionLeftLocal_z',
       'EyeDirectionLeftLocal_x', 'EyeDirectionLeftLocal_y',
       'EyeDirectionLeftLocal_z', 'EyePositionRightWorld_x',
       'EyePositionRightWorld_y', 'EyePositionRightWorld_z',
       'EyePositionRightLocal_x', 'EyePositionRightLocal_y',
       'EyePositionRightLocal_z', 'EyeDirectionRightWorld_x',
       'EyeDirectionRightWorld_y', 'EyeDirectionRightWorld_z',
       'EyeDirectionRightLocal_x', 'EyeDirectionRightLocal_y',
       'EyeDirectionRightLocal_z', 'EyePositionWorldCombinedTobii_x',
       'EyePositionWorldCombinedTobii_y', 'EyePositionWorldCombinedTobii_z',
       'EyePositionLocalCombinedTobii_x', 'EyePositionLocalCombinedTobii_y',
       'EyePositionLocalCombinedTobii_z', 'EyeDirectionWorldCombinedTobii_x',
       'EyeDirectionWorldCombinedTobii_y', 'EyeDirectionWorldCombinedTobii_z',
       'EyeDirectionLocalCombinedTobii_x', 'EyeDirectionLocalCombinedTobii_y',
       'EyeDirectionLocalCombinedTobii_z'],axis=1)
                
                et_df = pd.concat([et_df, tmpdf], ignore_index=True)
                   
    if not et_df.empty:
        et_df = et_df.drop_duplicates()
        et_df['timestamp'] = pd.to_datetime(et_df.UnixTimeStamp, utc=True, unit='s', origin='unix')
        et_df = et_df.sort_values(by=['timestamp'])
        et_df = et_df.reset_index(drop=True)
        et_df['is_blink'] = ((et_df['EyeOpennessLeftSranipal'] < 1)&(et_df['EyeOpennessRightSranipal'] < 1))
        path = os.path.join(PROCESSED_DATA_PATH,df['ParticipantUuid'][i]+'.csv')
        print(100/df['ParticipantUuid'].count()*(i+1),"%")
        et_df.to_csv(path)

100.0 %


In [6]:
#et_df.columns

In [7]:
files = glob.glob(f'{PROCESSED_DATA_PATH}\*.csv')
et_int_df = pd.DataFrame()
et_int = pd.DataFrame()
for fi in files[:1]:
    et_int_df = pd.read_csv(fi)
    
    et_int_df.loc[(et_int_df['is_blink'] == True),['EyeDirectionCombinedLocal_x', 'EyeDirectionCombinedLocal_y', 'EyeDirectionCombinedLocal_z']] = np.nan
    tmpdf = (
    et_int_df
    .groupby(['uid', 'scene'])['EyeDirectionCombinedLocal_x', 'EyeDirectionCombinedLocal_y', 'EyeDirectionCombinedLocal_z']
    .transform(lambda x: 
               x.interpolate(method='polynomial', order=3, limit_direction ='both'))
    .reset_index()
    )
    et_int_df[['combinedEyeDir_x_intp', 'combinedEyeDir_y_intp', 'combinedEyeDir_z_intp']] = tmpdf[['EyeDirectionCombinedLocal_x', 'EyeDirectionCombinedLocal_y', 'EyeDirectionCombinedLocal_z']]
    
    #et_int_df)
    #display(et_int_df.head(584)['EyeDirectionCombinedLocal_x'])
    display(et_int_df.loc[et_int_df['is_blink'] == True])
    et_int_df[['EyeDirectionCombinedLocal_x','timestamp']].plot()

C:\Users\MAXPIN~1\AppData\Local\Temp/ipykernel_4900/3900107164.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  et_int_df


,Unnamed: 0,UnixTimeStamp,TobiiTimeStamp,EyeOpennessLeftSranipal,EyeOpennessRightSranipal,HmdPosition_x,HmdPosition_y,HmdPosition_z,NoseVector_x,NoseVector_y,...,CarRotation_y,CarRotation_z,CarRotation_w,uid,scene,timestamp,is_blink,combinedEyeDir_x_intp,combinedEyeDir_y_intp,combinedEyeDir_z_intp
31,31,1.636207e+09,12376.541016,0.638555,0.827069,-273.167572,386.382538,-496.569000,-0.391136,-0.009702,...,-0.220602,0.000085,0.975359,489749a7e3394cdda22f5f2717a340a8,TrainingScene,2021-11-06 13:51:12.589849088+00:00,True,-0.054388,-0.004982,0.998507
32,32,1.636207e+09,0.000000,0.230945,0.088333,-273.167725,386.382629,-496.569244,-0.392212,-0.008865,...,-0.220602,0.000085,0.975359,489749a7e3394cdda22f5f2717a340a8,TrainingScene,2021-11-06 13:51:12.612787456+00:00,True,-0.056966,-0.003002,0.998343
33,33,1.636207e+09,0.000000,0.230945,0.088333,-273.167816,386.382660,-496.569336,-0.392771,-0.008741,...,-0.220602,0.000086,0.975359,489749a7e3394cdda22f5f2717a340a8,TrainingScene,2021-11-06 13:51:12.623758592+00:00,True,-0.056530,0.001587,0.998307
34,34,1.636207e+09,0.000000,0.138248,0.095724,-273.168060,386.382721,-496.569641,-0.394299,-0.008037,...,-0.220602,0.000085,0.975359,489749a7e3394cdda22f5f2717a340a8,TrainingScene,2021-11-06 13:51:12.645699584+00:00,True,-0.053809,0.007989,0.998369
35,35,1.636207e+09,0.000000,0.138248,0.095724,-273.168121,386.382690,-496.569702,-0.395856,-0.007914,...,-0.220602,0.000085,0.975359,489749a7e3394cdda22f5f2717a340a8,TrainingScene,2021-11-06 13:51:12.668638720+00:00,True,-0.049531,0.015406,0.998502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48732,48732,1.636208e+09,13273.514648,0.850338,0.915034,-1499.260986,131.253998,2380.560791,0.646624,0.002141,...,0.358106,-0.007078,0.933319,489749a7e3394cdda22f5f2717a340a8,CountryRoad,2021-11-06 14:06:09.564778240+00:00,True,NaN,NaN,NaN
48733,48733,1.636208e+09,13273.526367,0.850338,0.915034,-1498.994995,131.230194,2380.860840,0.645681,0.002768,...,0.358106,-0.007078,0.933319,489749a7e3394cdda22f5f2717a340a8,CountryRoad,2021-11-06 14:06:09.574750464+00:00,True,NaN,NaN,NaN
48734,48734,1.636208e+09,13273.547852,0.872943,0.925639,-1498.728027,131.206619,2381.160889,0.643250,0.002849,...,0.357757,-0.006889,0.933465,489749a7e3394cdda22f5f2717a340a8,CountryRoad,2021-11-06 14:06:09.597690112+00:00,True,NaN,NaN,NaN
48735,48735,1.636208e+09,13273.559570,0.872943,0.925639,-1498.461060,131.182968,2381.461426,0.642042,0.003018,...,0.357757,-0.006889,0.933465,489749a7e3394cdda22f5f2717a340a8,CountryRoad,2021-11-06 14:06:09.608660992+00:00,True,NaN,NaN,NaN


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …